## Descripción del modelo

### Objetivo

Nuestro objetivo es implementar una simulación realista de la Avenida General Paz. En los tramos de Liniers a Lugones. 

### Entidades / Agentes

Las entidades del modelo, qué serán los qué circulen por él, en nuestro modelo corresponden a los automóviles con sus respectivos conductores. Los agentes son autónomos, es decir, están dotados con comportamientos que les permiten tomar decisiones independientes.

### Atributos

Las entidades tienen distintos atributos que lo describen como:

- Posición **(x)**.
- Velocidad **(v)**.
- Velocidad máxima del vehículo **(vmax)**.
- Longitud del auto. **(l)**.
- Aceleración y desaceleración (cuanto tarda el vehículo en promedio en llegar de 0 a 80 y de 80 a 0 km/h) **(a)**.
- Tiempo de permanencia en la autopista **(t)**.
- Tiempo de reacción de frenado. **(tr)**

##### Interacción con otros agentes
Cada agente tiene un vehiculo por detras y por delante. Del vehiculo de adelante dependerá si el agente frena o no. Las reducciones de velocidad son las que causarán los *shockwaves*. 

- Distancia entre vehículo de atras **(bd)**.
- Distancia entre vehiculo de adelante **(fd)**. 

##### Comportamiento
- Los agentes reaccionan (mantienen su velocidad, aceleran y desaceleran) dependiendo de el ambiente y su vecindario. 
- Un agente podrá acelerar/desacelerar si el auto de adelante se lo permite. 
- Dependiendo de el comportamiento del agente que maneje el vehiculo, el mismo:
  -  respetará los limites de velocidad o no.
  -   tendrá mas o menos probabilidad de choque.


### Ambiente

El ambiente de nuestro agent-based transportation model será la infraestructura y capacidades de los nodos y enlaces de la red vial. Estas capacidades crean efectos de congestión (velocidades de viaje reducidas) limitando el número de agentes que se desplazan a través de la red de transporte en un momento dado.

### Variables de Decisión

Las variables de decisión son las qué generarán distintas alternativas o escenarios en el modelo. Es decir, estas serán las qué ayudarán a determinar cuál es el layout óptimo para la solución.
- Agregar peaje o no
- Agregar semáforos y cuantos.
- Agregar cámaras de fotomultas y cuantas.

### Variables de Referencia

Las variables de referencia son las obtenidas cómo resultado de la simulación. Se buscará mejorarlas en función al objetivo.
Tiempo promedio de recorrido de los vehículos.

### Variables de Estado

Las variables de estado son aquellas qué contienen la información necesaria para describir la situación actual de los recursos.
- Tasa de Ocupación en las principales zonas de aglomeración.
- Tasa de ocupación ocasionada por accidentes, vehículos detenidos, obras en la ruta, policías parando, etc.
- Tasa de ocupación ocasionada por mal tiempo.
- Tasa de ocupación del peaje o semáforos  (si es que se agregan).

### Recursos

- Los recursos utilizados en el modelo son:
- Las cámaras
- Los peajes 
- Los carteles de velocidad / Avisos. 

### Distribuciones

#### Distribución de arribos de vehiculos

Optado por la utilización de una *distribución exponencial negativa* como modelo para representar los arribos de vehículos, y esta elección se fundamenta en diversas consideraciones clave.

La distribución exponencial negativa es una selección frecuentemente empleada en la modelación de procesos de llegada aleatoria. En el contexto de rutas de vehículos, esta distribución se ajusta adecuadamente cuando se asume que los arribos de vehículos son eventos independientes y se producen a una tasa constante a lo largo del tiempo. En otras palabras, la probabilidad de que un vehículo llegue en un periodo determinado es constante y no depende del momento en que llegaron los vehículos previos.

Sin embargo, es fundamental destacar que emplearemos parámetros diferentes para la distribución, dependiendo de la franja horaria en la que nos encontremos, ya sea durante las horas pico o en otros momentos del día.

La distribución que elegiremos para modelar los arribos será la siguiente:
$$f(x | \lambda) = e^{(-\lambda x)}$$

$$
\lambda =
\begin{cases}
\frac{1}{\mu_1} = \frac{1}{15 \text{s}} \, & \text{si es hora pico} \\
\frac{1}{\mu_2} = \frac{1}{45 \text{s}}, & \text{si no es hora pico}
\end{cases}
$$

#### Distribución de velocidad

La distribución triangular es una elección adecuada para simular la velocidad de los autos en una ruta con un mínimo de 30 km/h, una moda de 70 km/h y un máximo de 100 km/h por lo siguiente:

Es una distribución continua que permite modelar situaciones en las que existe un valor mínimo, un valor máximo y un valor modal. En el contexto de la velocidad de los autos en una carretera, es razonable suponer que la mayoría de los autos se moverán a una velocidad cercana a la moda (70 km/h) y que habrá algunos que irán más rápido o más lento. Esto se ajusta bien a la forma de la distribución triangular.

$$
f(x) = \begin{cases}
0 & \text{si } x < 30 \\
\frac{2(x-30)}{(70-30)(100-30)} & \text{si } 30 \leq x < 70 \\
\frac{2(70-x)}{(70-30)(70-100)} & \text{si } 100 \leq x \leq 70 \\
0 & \text{si } x > 100
\end{cases}
$$

#### Distribución de variación de velocidad

En nuestra simulación de la Gral. Paz, cada agente se comporta de manera única y puede variar en su velocidad debido a diferentes factores, como la relación con otros agentes y las condiciones del entorno. Para modelar esta variación de velocidad, inicialmente utilizamos la distribución triangular, que nos proporciona una velocidad modal estimada para cada agente.

Sin embargo, sabemos que los agentes no mantendrán una velocidad constante en la carretera, ya que pueden optar por no respetar las leyes de velocidad, frenar de manera abrupta o ajustar su velocidad de acuerdo con la situación. Por lo tanto, para capturar esta variabilidad en la velocidad de manera más realista, decidimos emplear la distribución normal.

La distribución normal se seleccionó como una opción apropiada para modelar la variación de velocidad de los agentes debido a sus ventajas. Esta distribución es ampliamente reconocida por su capacidad para representar la variabilidad en datos del mundo real. Además, su simetría y versatilidad permiten modelar la idea de que algunos agentes pueden conducir a velocidades más altas o más bajas que la velocidad modal, mientras que la mayoría se mantendrá cerca de esta velocidad central.

La distribución normal también se ajusta fácilmente mediante la variación de la varianza, lo que nos permite controlar el grado de variación en las velocidades de los agentes de acuerdo con las necesidades de nuestra simulación.

$$
f(x) = \frac{1}{\sigma \sqrt{2\pi}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}
$$

$$
\begin{align*}
\mu & \text{ es la velocidad del agente obtenida a partir de la distribución triangular}, \\
\sigma & = 5 \text{ km/h} \text{ (varianza)}.
\end{align*}
$$


#### Distribución de frenado

Elegimos la *distribución exponencial* como modelo para la fuerza de frenado de los vehículos en función de su proximidad al vehículo de adelante debido a varias razones fundamentales:

1. **Falta de Memoria a Largo Plazo:** Esta propiedad implica que la probabilidad de que un vehículo frene con fuerza en un momento dado no depende de cuánto tiempo haya transcurrido desde el último frenado. En otras palabras, cada vehículo toma decisiones de frenado independientes en función de su proximidad actual al vehículo de adelante. Esta falta de dependencia temporal es esencial para capturar situaciones realistas en el tráfico, donde los eventos de frenado ocurren de manera estocástica y no están vinculados a eventos previos.

2. **Ajuste de la "Agresividad" del Frenado:** La distribución exponencial nos brinda la flexibilidad de ajustar el grado de "agresividad" en el frenado de los vehículos a través del parámetro de tasa (λ). Al variar el valor de λ, podemos modelar diferentes comportamientos de frenado. Un valor más alto de λ indica que los vehículos son menos propensos a frenar con fuerza cuando están cerca del vehículo de adelante, lo que podría representar situaciones de tráfico más fluidas. Por otro lado, un valor más bajo de λ indica una mayor probabilidad de frenado en distancias cercanas, lo que podría reflejar situaciones de tráfico más congestionadas o conductores más cautelosos.

3. **Modelización de Eventos Independientes:** La distribución exponencial es ampliamente utilizada para modelar eventos aleatorios que ocurren de manera independiente a lo largo del tiempo. En el contexto del tráfico, asumimos que los frenados de los vehículos son eventos estocásticos e independientes, ya que las decisiones de frenado de un vehículo no están influenciadas por los eventos de frenado de otros vehículos en el pasado. Esta suposición es coherente con la variabilidad inherente a las interacciones de tráfico en situaciones reales.

La formula que utilizaremos será la siguiente:
$$f(x | \lambda) = 1 - e^{(-\lambda x)}$$

$$
\lambda =
\begin{cases}
\frac{1}{\mu_1} = \frac{1}{1s}, & \text{ si } \text{dist\_to\_front\_agent()} ≤ 10 \text{m} \\
\frac{1}{\mu_2} = \frac{1}{3s}, & \text{ si } \text{dist\_to\_front\_agent()} > 10 \text{m}
\end{cases}
$$



#### Distribución del tiempo de reacción del agente

Elegimos la *distribución normal* para simular el tiempo de reacción debido a su capacidad para representar la variabilidad realista en los tiempos de reacción de las personas al frenar. La distribución normal se ajusta bien a situaciones en las que la mayoría de los individuos tienen tiempos de reacción cercanos a un valor promedio (la media), con menos personas que experimentan tiempos de reacción significativamente más cortos o más largos. Esto refleja con precisión cómo la mayoría de las personas conducen con tiempos de reacción similares, pero aún existe variabilidad natural. 

Luego de una busqueda de estudios de investigación acerca del comportamiento de los conductores en la autopista Gral. Paz y otras autopistas de Buenos Aires, encontramos que la media de el tiempo de reacción se aproxima a 1s con un desvio de 0.2s.

$$
f(x) = \frac{1}{\sigma \sqrt{2\pi}} e^{-\frac{(x-\mu)^2}{2\sigma^2}}
$$

$$
\begin{align*}
\mu & = 1s \\
\sigma & = 0.2s
\end{align*}
$$

## Simulación

In [1]:
import random 
from typing import Optional
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np

import pygame
import sys

pygame 2.5.1 (SDL 2.28.2, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
class Agent(pygame.sprite.Sprite):

    def __init__(self):
        self.position = 0                                             #posición inicial, todos los autos entran al inicio de la autopista
        self.max_velocity = 80/3.6                                    #velocidad maxima, todos los autos tienen la misma velocidad maxima (sabemos que en la Gral. Paz es de 80 km/h)  
        self.length = 5                                               #longitud del vehiculo, todos los autos tienen la misma longitud 
        self.front_agent = None                                       #agente de adelante
        self.back_agent = None                                        #agente de atras
        
        '''
        velocidad de agente es generada a partir de la distribución triangular con los siguientes parametros:
        - 30,000 m/s: velocidad minima
        - 100,000 m/s: velocidad maxima
        - 70,000 m/s: moda
        '''
        self.velocity = random.triangular(30/3.6, 100/3.6, 70/3.6)
        self.time = 0
        self.arrival_time = 0
    
        self.reaction_time = random.normalvariate(1, 0.2) 
    
    def __str__(self):
        front_agent_info = (
            f"(t={self.front_agent.time}, x={self.front_agent.position}, "
            f"v={self.front_agent.velocity})"
            if self.front_agent is not None else "None"
        )
        
        back_agent_info = (
            f"(t={self.back_agent.time}, x={self.back_agent.position}, "
            f"v={self.back_agent.velocity})"
            if self.back_agent is not None else "None"
        )
        
        return (
            f"Agent(t={self.time}, at={self.arrival_time}, x={self.position}, "
            f"v={self.velocity}, rt={self.reaction_time}, "
            f"fa={front_agent_info}, ba={back_agent_info})"
        )

    def dist_to_front_agent(self):
        if self.front_agent == None:
            return None

        else: #hay otro agente adelante
            return abs(self.position - self.front_agent.position - self.length)


    def dist_to_back_agent(self):
        if self.back_agent == None:
            return None

        else: #hay otro agent atras
            return abs(self.back_agent.position - self.position - self.back_agent.length)

    def speed_variation(self):
        if self.dist_to_front_agent() == None or self.dist_to_front_agent() >= 20:
            self.velocity = random.normalvariate(self.velocity, 5/3.6)


    def reduce_speed(self):
        # si se encuentra "cerca" del agente de adelante, reducirá su velocidad mediante una distribución exponencial con media 1.
        if self.dist_to_front_agent() is not None and self.dist_to_front_agent() < 10:
            reduction = (self.velocity - self.front_agent.velocity) / (1 - stats.expon.rvs(1/1))
            self.velocity -= reduction  # Reducir la velocidad
            
        # si se encuentra "lejos" del agente de adelante, reducirá su velocidad mediante una distribución exponencial con media 3.
        elif self.dist_to_front_agent() is not None and 10 <= self.dist_to_front_agent() <= 20:
            reduction = (self.velocity - self.front_agent.velocity) /(1 - stats.expon.rvs(1/3))
            self.velocity -= reduction  # Reducir la velocidad


    def move(self):
        self.position = self.position + self.velocity * 1 
    
    def get_position(self):
        return self.position

    def update(self):
        self.move()
        self.reduce_speed()
        self.speed_variation()

In [3]:
class Road:
    
    def __init__(self, m:int, c:bool):
        self.km = m
        self.agents = []
        self.time = 0
        self.total_time = 700
        self.congestion = c
    
    def add_agent(self, rate):
        # Si no hay agentes en la Gral.Paz, agregamos el primer agente
        if not self.agents:
            print("Agregando primer agente...")
            self.agents.append(Agent())
            print("Primer agente agregado: " + str(self.agents[0]))
            
            self.time_to_next_arrival = stats.expon.rvs(scale=1/rate)

        else:
            # Verificamos el tiempo desde el último agente
            if self.time - self.agents[-1].arrival_time > self.time_to_next_arrival:
                print("Agregando agente nro. ", len(self.agents))
                # Agregamos un nuevo agente a la lista
                new_agent = Agent()
                self.agents.append(new_agent)
                new_agent.arrival_time = self.time
                self.time_to_next_arrival = stats.expon.rvs(scale=1/rate)
                
                if len(self.agents) == 2:
                    position_new_agent = self.agents.index(new_agent)
                    new_agent.front_agent = self.agents[position_new_agent - 1]
                    self.agents[position_new_agent - 1].back_agent = new_agent

                # Configuramos el front y back car para el nuevo agente
                elif len(self.agents) >= 2:
                    position_new_agent = self.agents.index(new_agent)
                    new_agent.front_agent = self.agents[position_new_agent - 1]
                    self.agents[position_new_agent - 1].back_agent = new_agent

        self.update_road()

    def time_lapse(self):
        while self.time < self.total_time:
            if self.congestion == True:
                # Agrega autos con distribución exponencial negativa con media = 15
                self.add_agent(15)
            else:
                # Agrega autos con distribución exponencial negativa con media = 45
                self.add_agent(45)
        
    
    def update_road(self):
        self.time += 1
        for agent in self.agents:
            agent.update()
            print("Estado agente nro:", self.agents.index(agent), "->", agent)
            # print("Posición del agente:", self.agents.index(agent), "->", agent.get_position())
            
            #Si el agente llega al final de la carretera lo eliminamos
            if agent.get_position() > self.km:
                self.agents.remove(agent)

            #Reasignamos el front y back car de los agentes extremos de la lista
            if len(self.agents) > 0:
                self.agents[0].front_agent = None

        self.time_lapse()
        
    def plot(self):
        positions = []  # Lista para almacenar las posiciones de los agentes
        for agent in self.agents:
            positions.append(agent.get_position())  # Almacena la posición del agente en la lista

        return positions  # Devuelve la lista de posiciones para la animación

In [ ]:
road = Road(m=25000, c=True)
road.time_lapse()

In [5]:
import pygame
import sys

# Inicializar Pygame
pygame.init()

# Configuración de la ventana
width, height = 1000, 200
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Avenida General Paz")

# Colores
white = (255, 255, 255)
gray = (200, 200, 200)
green = (0, 150, 0)
black = (0, 0, 0)

# Crear una carretera
general_paz = Road(1000, True)

# Bucle principal
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Actualizar la carretera y los agentes en cada iteración
    general_paz.update_road()

   # Dibujar el pasto verde arriba de la carretera
    pygame.draw.rect(screen, green, (0, 0, width, height // 3))

    # Carretera gris
    pygame.draw.rect(screen, gray, (0, height // 3, width, height // 3))

    # Dibujar el pasto verde abajo de la carretera
    pygame.draw.rect(screen, green, (0, 2 * height // 3, width, height // 3))


    # Líneas blancas en el medio de la carretera
    line_width = 10
    line_height = 5
    num_lines = 30
    gap = width // (num_lines + 1)
    for i in range(num_lines):
        x = (i + 1) * gap - line_width // 2
        pygame.draw.rect(screen, white, (x, height // 2 - line_height // 2, line_width, line_height))


# Dibujar los agentes en la carretera
    for agent in general_paz.agents:
        pygame.draw.rect(screen, black, (agent.get_position(), height // 3 - 10, agent.length, 20))

    pygame.display.flip()

# Salir del juego
pygame.quit()
sys.exit()
